# ⚠️ IMPORTANT LEGAL DISCLAIMER

This code is provided for EDUCATIONAL and RESEARCH purposes only.
Song lyrics are the intellectual property of their respective copyright holders.

By using this code, you acknowledge that:
1. You will use the scraped content only for non-commercial, educational analysis
2. You will not redistribute or republish the lyrics
3. You understand that web scraping may violate website terms of service
4. This project falls under Fair Use doctrine for academic research
5. The author of this code is not responsible for any misuse

For production use, please obtain proper licensing through official APIs
or contact the copyright holders directly.

Taylor Swift's lyrics © Taylor Swift and respective publishers

In [1]:
import subprocess
import whisper
import json
import os
import pickle
from datetime import datetime
from yt_dlp import YoutubeDL
from pyannote.audio import Pipeline
from pyannote.core import Annotation, Segment
import glob

Check if ffmpeg is installed on the laptop. If not, it needs to be installed.

In [2]:
try:
    subprocess.run(["ffmpeg", "-version"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("FFmpeg is installed and available.")
except FileNotFoundError:
    print("FFmpeg is not installed or not found in PATH.")

FFmpeg is installed and available.


## Getting audio from YouTube link
The first step of the analysis is to extract audio from the podcast.

In [3]:
def download_audio_from_youtube(url: str, out_dir: str = "audio", sample_rate: int = 16000) -> str:
    """
    Downloads audio from YouTube videos and converts it to WAV (mono, 16kHz)
    Returns the path to the resulting file.
    """
    os.makedirs(out_dir, exist_ok=True)

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(out_dir, "%(id)s.%(ext)s"),
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "wav",
            "preferredquality": "192",
        }],
        "postprocessor_args": ["-ar", str(sample_rate), "-ac", "1"],
        "prefer_ffmpeg": True,
        "quiet": False,
        "no_warnings": True,
    }

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    video_id = info.get("id")
    pattern = os.path.join(out_dir, f"{video_id}*.wav")
    matches = glob.glob(pattern)
    if not matches:
        raise FileNotFoundError(f"No .wav file found for id={video_id}")
    return matches[0]

In [4]:
youtube_url = "https://www.youtube.com/watch?v=M2lX9XESvDE"
audio_path = download_audio_from_youtube(youtube_url)
print(f"Audio saved in: {audio_path}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=M2lX9XESvDE
[youtube] M2lX9XESvDE: Downloading webpage
[youtube] M2lX9XESvDE: Downloading tv client config
[youtube] M2lX9XESvDE: Downloading player 6742b2b9-main
[youtube] M2lX9XESvDE: Downloading tv player API JSON
[youtube] M2lX9XESvDE: Downloading ios player API JSON
[youtube] M2lX9XESvDE: Downloading m3u8 information
[info] Testing format 234
[info] M2lX9XESvDE: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1375
[download] Destination: audio/M2lX9XESvDE.mp4
[download] 100% of  115.45MiB in 00:04:07 at 476.97KiB/s                   
[ExtractAudio] Destination: audio/M2lX9XESvDE.wav
Deleting original file audio/M2lX9XESvDE.mp4 (pass -k to keep)
Audio saved in: audio/M2lX9XESvDE.wav


## Getting text from audio file
The second step is to convert audio to text.

In [5]:
# Transcribe audio with Whisper
def transcribe_audio_whisper(audio_path: str, model_size="small"):
    print(f"[INFO] Loading Whisper model: {model_size}")
    model = whisper.load_model(model_size)  # "small", "medium", or "large" for better accuracy
    print("[INFO] Starting transcription...")
    result = model.transcribe(audio_path, verbose=True)
    return result  # Contains 'segments' with text, start, end


In [10]:
# model_size is set to 'medium' because results from 'small' were very poor, but 'large' takes too long to process

transcription = transcribe_audio_whisper(audio_path, 'medium')

[INFO] Loading Whisper model: medium


100%|█████████████████████████████████████| 1.42G/1.42G [00:59<00:00, 25.5MiB/s]


[INFO] Starting transcription...


/Users/Natalie/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:08.220]  All right, let's get to the part of this show that I think is what everybody is going to
[00:08.220 --> 00:09.220]  be talking about.
[00:09.220 --> 00:10.220]  Do I get to say it?
[00:10.220 --> 00:13.020]  Do I get to say I get to say the two words?
[00:13.020 --> 00:14.020]  Yes.
[00:14.020 --> 00:15.020]  First of all, you can do whatever you want to.
[00:15.020 --> 00:16.020]  I don't know.
[00:16.020 --> 00:17.020]  We're not going to review.
[00:17.020 --> 00:19.740]  This is very much whatever within the framework of the podcast.
[00:19.740 --> 00:21.900]  I'm a fan of the podcast.
[00:21.900 --> 00:25.060]  Typically allow the guests to say new news.
[00:25.060 --> 00:26.340]  Yeah, I want to do it.
[00:26.340 --> 00:34.800]  I think Taylor has a little bit of new news.
[00:34.800 --> 00:36.440]  Welcome back to New Heights.
[00:36

...

[01:44:02.360 --> 01:44:09.240]  But you had to jump through 50 million hoops in this obstacle course that is your show.
[01:44:09.960 --> 01:44:12.520]  And you did it.
[01:44:12.520 --> 01:44:14.040]  You got two more in a row.
[01:44:15.080 --> 01:44:16.120]  But you did it tonight.
[01:44:16.920 --> 01:44:17.880]  That's all that matters.
[01:44:17.880 --> 01:44:22.760]  And the reason I wanted to have it sort of like an offstage moment as the main album cover is
[01:44:22.760 --> 01:44:26.920]  because this album isn't really about what happened to me on stage.
[01:44:26.920 --> 01:44:30.600]  It's about what I was going through offstage.
[01:44:31.560 --> 01:44:32.060]  Sure.
[01:44:32.600 --> 01:44:35.960]  It's like, you know, I didn't want to have like the lights are bright.
[01:44:35.960 --> 01:44:37.720]  I'm on the stage as the main album cover.
[01:44:37.720 --> 01:44:44.440]  It's just this to me tells more of what the actual contents lyrically of the album are.
[01:44:44

[01:48:47.320 --> 01:48:48.520]  The cat that we had.
[01:48:48.520 --> 01:48:49.320]  What did you think of?
[01:48:49.320 --> 01:48:51.000]  What did you think of my three cats?
[01:48:51.000 --> 01:48:51.640]  My three perfect.
[01:48:51.640 --> 01:48:53.720]  The only one that would go near me was Benjamin.
[01:48:55.160 --> 01:48:55.320]  Yeah.
[01:48:55.320 --> 01:48:58.120]  Cause the other two consents that you had resting cat.
[01:48:58.120 --> 01:49:00.360]  He turned that might be it.
[01:49:00.920 --> 01:49:01.720]  All right.
[01:49:01.720 --> 01:49:06.600]  If you had to choose to bake one thing to show off your baking skills, what would you choose?
[01:49:06.600 --> 01:49:10.280]  Oh, right now it would be my cinnamon swirl sourdough.
[01:49:11.160 --> 01:49:12.920]  Cinnamon swirl sourdough.
[01:49:12.920 --> 01:49:14.280]  I thought you were going to go pop tarts.
[01:49:14.280 --> 01:49:15.400]  Pop tarts, pop tarts.
[01:49:15.400 --> 01:49:17.640]  It depends on who 

[01:53:31.640 --> 01:53:31.880]  Yeah.
[01:53:31.880 --> 01:53:32.520]  You need to live a little.
[01:53:32.520 --> 01:53:33.560]  You're doing it wrong.
[01:53:33.560 --> 01:53:34.120]  Live a little.
[01:53:35.080 --> 01:53:36.840]  Get into a fake mop cart.
[01:53:36.840 --> 01:53:40.040]  Everyone knows you're in there, but you're in there like, I'm sneaking around.
[01:53:40.600 --> 01:53:41.640]  Nobody knows a minute.
[01:53:41.640 --> 01:53:42.600]  Everyone knows you're in there.
[01:53:45.640 --> 01:53:49.080]  Well, it's like, it's one of those things like I work so hard to try to surprise fans.
[01:53:49.640 --> 01:53:51.880]  And they're like, I don't want to be surprised.
[01:53:51.880 --> 01:53:54.120]  Sometimes they really don't think they want to be surprised.
[01:53:54.120 --> 01:53:55.400]  I want to figure stuff out.
[01:53:55.400 --> 01:53:56.600]  No, they do.
[01:53:56.600 --> 01:54:01.720]  But then I know, I know that when I can really get them and surprise t

[01:58:15.320 --> 01:58:21.320]  So it was not the, like the altitude I was reaching, there was more pressure every step of the way,
[01:58:21.320 --> 01:58:23.960]  but I was getting more creative control and freedom every time.
[01:58:24.920 --> 01:58:30.360]  And that is why it felt so much more special to keep working harder, to get to new levels.
[01:58:30.360 --> 01:58:36.280]  It's not like, oh, like I can afford to buy this or whatever, like it, not to discount that.
[01:58:37.000 --> 01:58:41.400]  But for me, every time I got higher up on a new rung of the ladder,
[01:58:41.400 --> 01:58:46.600]  I could make stuff in a more focused and free and autonomous way.
[01:58:46.600 --> 01:58:47.960]  What else would you want as an artist?
[01:58:47.960 --> 01:58:51.880]  I never really let myself be like, I've made it, but the arrows tour, I was like, oh,
[01:58:54.120 --> 01:58:55.160]  this is different.
[01:58:55.160 --> 01:58:59.080]  Like this is not, this is nothing like what 

[02:03:00.760 --> 02:03:07.880]  All right.
[02:03:07.880 --> 02:03:11.080]  I don't know where to go from here.
[02:03:15.080 --> 02:03:16.200]  There was confetti all over.
[02:03:20.520 --> 02:03:24.120]  Oh, this is like when the cats taste something they don't that they're confused by.
[02:03:25.560 --> 02:03:26.600]  This is my favorite impression.
[02:03:26.600 --> 02:03:27.160]  This is it.
[02:03:27.160 --> 02:03:28.040]  It's a confused.
[02:03:28.040 --> 02:03:28.840]  The cat ate something.
[02:03:28.840 --> 02:03:30.440]  It doesn't know what it is.
[02:03:30.520 --> 02:03:33.080]  This is this that gets gets me every time.
[02:03:33.080 --> 02:03:34.920]  That gets me every time, man.
[02:03:34.920 --> 02:03:36.200]  I will laugh at that.
[02:03:36.200 --> 02:03:40.200]  I will like fall off the couch laughing at that if he'll do it at, you know.


In [7]:
# Create directory for saved files (if it doesn't exist yet)
save_dir = "saved_diarizations" # using this name, so eventually all needed files were in one directory
os.makedirs(save_dir, exist_ok=True)

# Saving json file with results of transcription
transcription_file = os.path.join(save_dir, "transcription_medium.json")
with open(transcription_file, "w", encoding="utf-8") as f:
    json.dump(transcription, f, ensure_ascii=False, indent=2)

print(f"[SAVED] Transcription: {transcription_file}")

[SAVED] Transcription: saved_diarizations/transcription_medium.json


## Defining speakers
The next step is to assign the corresponding speaker to each text segment.

In [29]:
HF_TOKEN = "" #<- paste your hf tolen

print("[INFO] Loading speaker diarization pipeline...")

try:
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1", 
        use_auth_token=HF_TOKEN
    )
    print("[INFO] Successfully loaded speaker-diarization-3.1 model")
except Exception as e:
    print(f"[ERROR] Could not load v3.1, trying v3.0: {e}")
    try:
        pipeline = Pipeline.from_pretrained(
            "pyannote/speaker-diarization-3.0", 
            use_auth_token=HF_TOKEN
        )
        print("[INFO] Successfully loaded speaker-diarization-3.0 model")
    except Exception as fallback_e:
        print(f"[ERROR] All models failed to load: {fallback_e}")
        raise

print("[INFO] Running diarization on audio file...")
diarization = pipeline(audio_path)

# Print diarization results for debugging
print("[INFO] Diarization results:")
speakers = set()
for turn, _, speaker in diarization.itertracks(yield_label=True):
    speakers.add(speaker)
    print(f"  {turn.start:.2f}s - {turn.end:.2f}s: {speaker}")
print(f"[INFO] Detected {len(speakers)} speakers: {', '.join(sorted(speakers))}")

# Step 4: Merge transcription segments with diarization labels
def merge_transcript_with_speakers(transcript, diarization):
    """
    Merge transcription segments with speaker diarization labels.
    
    Args:
        transcript: Dictionary with 'segments' key containing transcription data
        diarization: pyannote diarization result
    
    Returns:
        List of merged segments with speaker labels
    """
    merged_segments = []
    
    for seg in transcript["segments"]:
        start, end, text = seg["start"], seg["end"], seg["text"]
        speaker_label = None
        max_overlap = 0
        
        # Find speaker label by maximum overlapping diarization segment
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            # Calculate overlap between transcript segment and diarization segment
            overlap_start = max(start, turn.start)
            overlap_end = min(end, turn.end)
            overlap_duration = max(0, overlap_end - overlap_start)
            
            if overlap_duration > max_overlap:
                max_overlap = overlap_duration
                speaker_label = speaker
        
        merged_segments.append({
            "start": start,
            "end": end,
            "text": text.strip(),
            "speaker": speaker_label,
            "overlap_duration": max_overlap
        })
    
    return merged_segments

# Assuming 'transcription' variable exists from previous steps
try:
    merged = merge_transcript_with_speakers(transcription, diarization)
    
    print("[INFO] Example of merged transcript:")
    for i, m in enumerate(merged[:5]):
        overlap_info = f"(overlap: {m['overlap_duration']:.2f}s)" if m['overlap_duration'] > 0 else "(no overlap)"
        print(f"  {i+1}. {m['speaker']}: {m['text']} {overlap_info}")
    
    # Step 5: Show available speakers and filter by chosen speaker
    available_speakers = set(m["speaker"] for m in merged if m["speaker"] is not None)
    print(f"[INFO] Available speakers in merged transcript: {', '.join(sorted(available_speakers))}")
    
    # Choose the most frequent speaker as default, or specify manually
    if available_speakers:
        speaker_counts = {}
        for m in merged:
            if m["speaker"]:
                speaker_counts[m["speaker"]] = speaker_counts.get(m["speaker"], 0) + 1
        
        # Get most frequent speaker
        most_frequent_speaker = max(speaker_counts, key=speaker_counts.get)
        target_speaker = most_frequent_speaker  # or specify manually like "SPEAKER_00"
        
        print(f"[INFO] Speaker statistics:")
        for speaker, count in sorted(speaker_counts.items()):
            print(f"  {speaker}: {count} segments")
        
        print(f"[INFO] Using most frequent speaker: {target_speaker}")
        
        # Filter by chosen speaker
        filtered_segments = [m for m in merged if m["speaker"] == target_speaker]
        filtered_text = [m["text"] for m in filtered_segments]
        
        print(f"[INFO] Filtered text for {target_speaker} ({len(filtered_segments)} segments):")
        combined_text = " ".join(filtered_text)
        
        # Print first 500 characters as preview
        preview_length = 500
        if len(combined_text) > preview_length:
            print(f"  {combined_text[:preview_length]}...")
            print(f"  [Total length: {len(combined_text)} characters]")
        else:
            print(f"  {combined_text}")
            
        # Optional: Save to file
        # with open(f"speaker_{target_speaker}_transcript.txt", "w", encoding="utf-8") as f:
        #     f.write(combined_text)
        # print(f"[INFO] Saved to speaker_{target_speaker}_transcript.txt")
        
    else:
        print("[WARNING] No speakers found in merged transcript!")
        
except NameError:
    print("[ERROR] 'transcription' variable not found. Make sure you have run the transcription step first.")
except Exception as e:
    print(f"[ERROR] Error during merging: {e}")
    print(f"[ERROR] Error type: {type(e).__name__}")

[INFO] Loading speaker diarization pipeline...


[INFO] Successfully loaded speaker-diarization-3.1 model
[INFO] Running diarization on audio file...


/Users/Natalie/anaconda3/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  std = sequences.std(dim=-1, correction=1)


[INFO] Diarization results:
  0.03s - 0.32s: SPEAKER_00
  0.91s - 2.33s: SPEAKER_00
  3.30s - 8.99s: SPEAKER_00
  9.24s - 13.14s: SPEAKER_04
  10.56s - 11.12s: SPEAKER_00
  11.59s - 12.42s: SPEAKER_00
  13.14s - 18.71s: SPEAKER_00
  15.18s - 15.20s: SPEAKER_04
  15.20s - 16.53s: SPEAKER_03
  17.80s - 22.36s: SPEAKER_03
  22.36s - 22.76s: SPEAKER_00
  22.39s - 22.64s: SPEAKER_03
  22.76s - 23.03s: SPEAKER_03
  23.03s - 26.27s: SPEAKER_00
  26.31s - 31.03s: SPEAKER_01
  34.29s - 53.42s: SPEAKER_01
  38.10s - 38.35s: SPEAKER_00
  38.35s - 38.64s: SPEAKER_02
  38.64s - 38.67s: SPEAKER_00
  38.67s - 38.83s: SPEAKER_02
  38.83s - 38.98s: SPEAKER_00
  44.72s - 45.04s: SPEAKER_00
  50.37s - 51.03s: SPEAKER_00
  53.61s - 64.97s: SPEAKER_01
  56.51s - 56.55s: SPEAKER_00
  65.64s - 70.30s: SPEAKER_01
  70.30s - 75.75s: SPEAKER_00
  76.37s - 76.69s: SPEAKER_00
  77.20s - 79.14s: SPEAKER_00
  77.30s - 77.94s: SPEAKER_01
  79.75s - 88.44s: SPEAKER_00
  89.28s - 92.61s: SPEAKER_00
  92.76s - 94.51s: 

[INFO] Example of merged transcript:
  1. SPEAKER_00: All right, let's get to the part of this show that I think is what everybody is going to (overlap: 4.92s)
  2. SPEAKER_00: be talking about. (overlap: 0.77s)
  3. SPEAKER_04: Do I get to say it? (overlap: 0.98s)
  4. SPEAKER_04: Do I get to say I get to say the two words? (overlap: 2.80s)
  5. SPEAKER_00: Yes. (overlap: 0.88s)
[INFO] Available speakers in merged transcript: SPEAKER_00, SPEAKER_01, SPEAKER_02, SPEAKER_03, SPEAKER_04
[INFO] Speaker statistics:
  SPEAKER_00: 588 segments
  SPEAKER_01: 431 segments
  SPEAKER_02: 12 segments
  SPEAKER_03: 1525 segments
  SPEAKER_04: 24 segments
[INFO] Using most frequent speaker: SPEAKER_03
[INFO] Filtered text for SPEAKER_03 (1525 segments):
  This is very much whatever within the framework of the podcast. I'm a fan of the podcast. What's that intro, Jason? Oh my God. I've seen this before. Look, his soul has left his body. No, that was so good. Thank you. Thank you for screaming for li

What we see in the data is that diarization identified 5 speakers, but we know there were only 3 speakers in the podcast.
So we need to fix it. Let's start by defining text samples for all speakers to determine where SPEAKER_02 and SPEAKER_04 should be merged.

In [39]:
# Show text samples for each speaker to manually identify who is who

speaker_texts = {}
speaker_counts = {}

# Collect text for each speaker
for segment in merged:
    speaker = segment["speaker"]
    if speaker is None:
        continue
    
    if speaker not in speaker_texts:
        speaker_texts[speaker] = ""
        speaker_counts[speaker] = 0
    
    speaker_texts[speaker] += " " + segment["text"]
    speaker_counts[speaker] += 1

# Display samples for each speaker
print("=" * 80)
print("SPEAKER TEXT SAMPLES FOR MANUAL IDENTIFICATION")
print("=" * 80)

for speaker in sorted(speaker_texts.keys()):
    text = speaker_texts[speaker].strip()
    count = speaker_counts[speaker]
    
    print(f"\n{speaker} ({count} segments):")
    print("-" * 50)
    
    # Show first 1500 characters
    if len(text) > 1500:
        display_text = text[:1500] + "..."
    else:
        display_text = text
        
    print(display_text)
    print("-" * 50)

print(f"\nSUMMARY:")
for speaker in sorted(speaker_counts.keys()):
    print(f"  {speaker}: {speaker_counts[speaker]} segments")
print("=" * 80)

SPEAKER TEXT SAMPLES FOR MANUAL IDENTIFICATION

SPEAKER_00 (588 segments):
--------------------------------------------------
All right, let's get to the part of this show that I think is what everybody is going to be talking about. Yes. First of all, you can do whatever you want to. I don't know. We're not going to review. Typically allow the guests to say new news. Yeah, I want to do it. That's right. 92 percenters. You may remember when we said New Heights wasn't coming back till August 27th. Well, that was a lie. That was a lie. And hopefully you can forgive us because we got a as Travis said, we have a very special episode today that we just simply could not turn down. That's right. This is a special preseason episode that we decided to bring to you a little early. Our guest today is the singer, songwriter, producer and director from Nashville, Tennessee. That's bullshit. She is from waiting. She is the most awarded artist in the history of the American Music Awards, Billboard Mus

## Fixing speakers [from 5 to 3]
Based on the samples, it's clear that the two redundant speakers should be part of TS segments, i.e., SPEAKER_03.
SPEAKER_00 is JK and SPEAKER_01 is TK.

In [45]:
# Correct speaker mapping and rename to real names
from pyannote.core import Annotation, Segment

# Create corrected diarization with proper speaker names
corrected_diarization = Annotation()

# Define speaker corrections and renaming
speaker_mapping = {
    "SPEAKER_00": "Jason Kelsey",
    "SPEAKER_01": "Travis Kelsey", 
    "SPEAKER_02": "Taylor Swift",  # merge to Taylor Swift
    "SPEAKER_03": "Taylor Swift",  # already Taylor Swift
    "SPEAKER_04": "Taylor Swift"   # merge to Taylor Swift
}

print("[INFO] Applying speaker corrections...")
print("[INFO] Mapping:")
for old, new in speaker_mapping.items():
    print(f"  {old} -> {new}")

# Apply corrections to diarization
for turn, _, speaker in diarization.itertracks(yield_label=True):
    # Get corrected speaker name
    corrected_speaker = speaker_mapping.get(speaker, speaker)
    
    # Add segment with corrected speaker name
    corrected_diarization[turn] = corrected_speaker

# Replace original diarization with corrected version
diarization = corrected_diarization

print("\n[INFO] Corrected diarization results:")
corrected_speakers = set()
corrected_stats = {}

# Show corrected results
for turn, _, speaker in diarization.itertracks(yield_label=True):
    corrected_speakers.add(speaker)
    corrected_stats[speaker] = corrected_stats.get(speaker, 0) + 1
    print(f"  {turn.start:.2f}s - {turn.end:.2f}s: {speaker}")

print(f"\n[INFO] Final speakers: {', '.join(sorted(corrected_speakers))}")
print("[INFO] Final speaker statistics:")
for speaker, count in sorted(corrected_stats.items()):
    print(f"  {speaker}: {count} segments")

print(f"\n[INFO] Total speakers after correction: {len(corrected_speakers)}")
print("[INFO] Diarization variable updated with correct speaker names")

[INFO] Applying speaker corrections...
[INFO] Mapping:
  SPEAKER_00 -> Jason Kelsey
  SPEAKER_01 -> Travis Kelsey
  SPEAKER_02 -> Taylor Swift
  SPEAKER_03 -> Taylor Swift
  SPEAKER_04 -> Taylor Swift

[INFO] Corrected diarization results:
  0.03s - 0.32s: Jason Kelsey
  0.91s - 2.33s: Jason Kelsey
  3.30s - 8.99s: Jason Kelsey
  9.24s - 13.14s: Taylor Swift
  10.56s - 11.12s: Jason Kelsey
  11.59s - 12.42s: Jason Kelsey
  13.14s - 18.71s: Jason Kelsey
  15.18s - 15.20s: Taylor Swift
  15.20s - 16.53s: Taylor Swift
  17.80s - 22.36s: Taylor Swift
  22.36s - 22.76s: Jason Kelsey
  22.39s - 22.64s: Taylor Swift
  22.76s - 23.03s: Taylor Swift
  23.03s - 26.27s: Jason Kelsey
  26.31s - 31.03s: Travis Kelsey
  34.29s - 53.42s: Travis Kelsey
  38.10s - 38.35s: Jason Kelsey
  38.35s - 38.64s: Taylor Swift
  38.64s - 38.67s: Jason Kelsey
  38.67s - 38.83s: Taylor Swift
  38.83s - 38.98s: Jason Kelsey
  44.72s - 45.04s: Jason Kelsey
  50.37s - 51.03s: Jason Kelsey
  53.61s - 64.97s: Travis Kel

### Saving file with fixed speakers
This is needed so we don't have to run all this preparation each time we want to analyze the data.

In [48]:
# Create directory for saved files
save_dir = "saved_diarizations"
os.makedirs(save_dir, exist_ok=True)

# Base filename
audio_filename = "podcast"  # change to your actual audio filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. Save as pickle (fastest loading for Python)
pickle_file = os.path.join(save_dir, f"{audio_filename}_corrected_diarization.pkl")
with open(pickle_file, 'wb') as f:
    pickle.dump(diarization, f)
print(f"[SAVED] Pickle format: {pickle_file}")

# 2. Save as JSON (universal format)
json_data = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    json_data.append({
        "start": float(turn.start),
        "end": float(turn.end),
        "speaker": speaker,
        "duration": float(turn.end - turn.start)
    })

json_file = os.path.join(save_dir, f"{audio_filename}_corrected_diarization.json")
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump({
        "audio_file": f"{audio_filename}.wav",
        "created_at": datetime.now().isoformat(),
        "speakers": sorted(list(set(item["speaker"] for item in json_data))),
        "total_segments": len(json_data),
        "segments": json_data
    }, f, indent=2, ensure_ascii=False)
print(f"[SAVED] JSON format: {json_file}")

# 3. Save as readable text
txt_file = os.path.join(save_dir, f"{audio_filename}_corrected_diarization.txt")
with open(txt_file, 'w', encoding='utf-8') as f:
    f.write(f"Corrected Diarization Results\n")
    f.write(f"Audio: {audio_filename}.wav\n")
    f.write(f"Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Speakers: Jason Kelsey, Travis Kelsey, Taylor Swift\n")
    f.write(f"Total segments: {len(json_data)}\n")
    f.write("=" * 60 + "\n\n")
    
    for item in json_data:
        f.write(f"{item['start']:>7.2f}s - {item['end']:>7.2f}s ({item['duration']:>5.2f}s): {item['speaker']}\n")
print(f"[SAVED] Text format: {txt_file}")

print(f"\n[SUCCESS] Diarization saved in 3 formats!")
print(f"[INFO] Files saved in: {save_dir}/")

# ============ LOADING FUNCTIONS ============

def load_diarization_from_pickle(pickle_file):
    """Load diarization from pickle file (fastest)"""
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)

def load_diarization_from_json(json_file):
    """Load diarization from JSON file (universal)"""
    from pyannote.core import Annotation, Segment
    
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    annotation = Annotation()
    for segment_data in data["segments"]:
        segment = Segment(segment_data["start"], segment_data["end"])
        annotation[segment] = segment_data["speaker"]
    
    return annotation

print(f"\n" + "=" * 70)
print("HOW TO LOAD IN FUTURE:")
print("=" * 70)
print("# Instead of running the whole pipeline again, just load:")
print(f"diarization = load_diarization_from_pickle('{pickle_file}')")
print("# Or from JSON:")
print(f"diarization = load_diarization_from_json('{json_file}')")
print("# Loading takes 1 second instead of 2+ hours!")
print("=" * 70)

[SAVED] Pickle format: saved_diarizations/podcast_corrected_diarization.pkl
[SAVED] JSON format: saved_diarizations/podcast_corrected_diarization.json
[SAVED] Text format: saved_diarizations/podcast_corrected_diarization.txt

[SUCCESS] Diarization saved in 3 formats!
[INFO] Files saved in: saved_diarizations/

HOW TO LOAD IN FUTURE:
# Instead of running the whole pipeline again, just load:
diarization = load_diarization_from_pickle('saved_diarizations/podcast_corrected_diarization.pkl')
# Or from JSON:
diarization = load_diarization_from_json('saved_diarizations/podcast_corrected_diarization.json')
# Loading takes 1 second instead of 2+ hours!


### Rebuild merged segments with corrected speaker names

In [51]:
# Rebuild merged segments with corrected speaker names

print("[INFO] Rebuilding merged segments with corrected speakers...")

# Clear old merged data and rebuild with corrected diarization
new_merged = []

for seg in transcription["segments"]:
    start, end, text = seg["start"], seg["end"], seg["text"]
    speaker_label = None
    max_overlap = 0
    
    # Find speaker label by maximum overlapping diarization segment
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        # Calculate overlap between transcript segment and diarization segment
        overlap_start = max(start, turn.start)
        overlap_end = min(end, turn.end)
        overlap_duration = max(0, overlap_end - overlap_start)
        
        if overlap_duration > max_overlap:
            max_overlap = overlap_duration
            speaker_label = speaker
    
    new_merged.append({
        "start": start,
        "end": end,
        "text": text.strip(),
        "speaker": speaker_label,
        "overlap_duration": max_overlap
    })

# Replace old merged with new one
merged = new_merged

print("[INFO] Updated merged segments with corrected speaker names")

# Show new statistics
speaker_stats = {}
for segment in merged:
    if segment["speaker"]:
        speaker_stats[segment["speaker"]] = speaker_stats.get(segment["speaker"], 0) + 1

print("[INFO] New merged statistics:")
for speaker, count in sorted(speaker_stats.items()):
    print(f"  {speaker}: {count} segments")

# Show sample of corrected merged data
print("\n[INFO] Sample of corrected merged segments:")
for i, segment in enumerate(merged[:5]):
    print(f"  {i+1}. {segment['speaker']}: \"{segment['text'][:80]}...\"")

print(f"\n[SUCCESS] Merged variable updated with {len(merged)} segments")

[INFO] Rebuilding merged segments with corrected speakers...
[INFO] Updated merged segments with corrected speaker names
[INFO] New merged statistics:
  Jason Kelsey: 588 segments
  Taylor Swift: 1561 segments
  Travis Kelsey: 431 segments

[INFO] Sample of corrected merged segments:
  1. Jason Kelsey: "All right, let's get to the part of this show that I think is what everybody is ..."
  2. Jason Kelsey: "be talking about...."
  3. Taylor Swift: "Do I get to say it?..."
  4. Taylor Swift: "Do I get to say I get to say the two words?..."
  5. Jason Kelsey: "Yes...."

[SUCCESS] Merged variable updated with 2584 segments
